In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import SGD,Adam
from keras.callbacks import CSVLogger
import json
import keras
import numpy as np
from keras.utils.np_utils import to_categorical
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib

In [3]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17520587514943627619
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15771998618
locality {
  bus_id: 1
}
incarnation: 16000391501392083131
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [5]:
docs = []
labels = []

In [6]:
num_classes = 102

In [7]:
def load_data(path1):
    temp_docs = []
    labels = []
    with open(path1) as json_file:
        data = json.load(json_file)
        
    for x in data:
        temp_docs.append(x['body'])
#         labels.append(keras.utils.to_categorical(np.array(list(x['middle_label'])),num_classes=102))
        labels.append((np.array(list(x['middle_label']))))
    len1 = len(temp_docs)
    print('number of human rights docs is: '+str(len1))
    return temp_docs,labels

In [8]:
docs,labels = load_data('/home/tigermlt/CS341/github_repo/CS341/data/data_with_middle_layer_and_label.json')

number of human rights docs is: 55326


In [9]:
# docs_train = docs[:45000]
# labels_train = labels[:45000]
# docs_val = docs[45000:50000]
# labels_val = labels[45000:50000]
# docs_test = docs[50000:]
# labels_test = labels[50000:]
# docs = docs[:100]
# labels = labels[:100]

In [10]:
print(docs[0])

The much-awaited debate on how international patent laws affect developing countries' access to medications begins Wednesday, Jun 20, at the World Trade Organisation (WTO), and the countries, corporations and civil society groups involved are putting the final touches on their arguments.


In [11]:
import random

In [12]:
# random shuffle the data
c = list(zip(docs, labels))

In [13]:
random.shuffle(c)

In [14]:
docs, labels = zip(*c)

In [15]:
docs = list(docs)
labels = list(labels)

In [16]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)

In [17]:
# compute max document length
max_val = -1
for i in range(len(docs)):
    temp_val = len(docs[i])
    if temp_val>max_val:
        max_val = temp_val
print(max_val)

15260


In [18]:
# pad documents to a max length, compute by calculating the maximum document length
# max_length = 15260
max_length = 8000
# max_length = 1068
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [19]:
print(len(padded_docs))

55326


In [20]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [21]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [22]:
# # prepare validation data

# # prepare tokenizer
# t_val = Tokenizer()
# t_val.fit_on_texts(docs_val)
# # integer encode the documents
# encoded_docs_val = t_val.texts_to_sequences(docs_val)

# padded_docs_val = pad_sequences(encoded_docs_val, maxlen=max_length, padding='post')

In [23]:
def build_model():
    # define model
    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False)
    model.add(e)
    model.add(LSTM(128,dropout = 0.3,return_sequences=True))
    model.add(LSTM(128,dropout = 0.3))
#     model.add(LSTM(128,dropout = 0.8,))
#     model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    return model

In [24]:
model = build_model()

In [25]:
# def euclidean_distance_loss(y_true, y_pred):
#     y_log = K.log(y_pred)
#     y_log_2 = K.log(1-y_pred)
    
#     part1 = -y_true*y_log
#     part2 = -(1-y_true)*y_log_2
    
#     matrix = part1 + part2
#     mean_row = K.mean(matrix,axis=1)
    
#     loss = K.mean(mean_row,axis=0)

#     return loss

In [26]:
def accuracy(y_true, y_pred):
    diff = K.abs(y_true-y_pred)
    correct_num = K.sum(tf.to_int32(K.greater(0.1,diff)))
    accuracy = correct_num/16/102
    return accuracy

In [28]:
csv_logger=CSVLogger('middle_class_v4.csv',append=True,separator=';')

In [29]:
# compile the model
model.compile(optimizer=Adam(lr=0.01), loss=['binary_crossentropy'], metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 8000, 300)         28578900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 8000, 128)         219648    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 102)               13158     
Total params: 28,943,290
Trainable params: 364,390
Non-trainable params: 28,578,900
_________________________________________________________________
None


In [30]:
print(padded_docs.shape)

(55326, 8000)


In [31]:
with tf.device('/device:GPU:0'):
    # fit the model
    model.fit(padded_docs, np.array(labels), epochs=3, validation_split = 0.10, batch_size = 128,callbacks=[csv_logger])

Train on 49793 samples, validate on 5533 samples
Epoch 1/3
49793/49793 [==============================] - 13641s 274ms/step - loss: 0.1413 - acc: 0.9522 - val_loss: 0.1476 - val_acc: 0.9415
Epoch 2/3
49793/49793 [==============================] - 13404s 269ms/step - loss: 0.1439 - acc: 0.9520 - val_loss: 0.1346 - val_acc: 0.9529
Epoch 3/3
49793/49793 [==============================] - 13602s 273ms/step - loss: 0.1351 - acc: 0.9525 - val_loss: 0.1339 - val_acc: 0.9529


In [32]:
model.save('middle_class_v4.h5')

In [ ]:
with tf.device('/device:GPU:0'):
    pred = model.predict(padded_docs)

In [ ]:
print(pred[0]>0.1)
print(labels[0]>0.1)

In [ ]:
docs_test = docs[15000:16000]
labels_test = labels[15000:16000]

In [ ]:
with open('/home/tigermlt/CS341/github_repo/CS341/data/10000.json') as json_file:
    data = json.load(json_file)
    for x in data:
        if x['content'] is not None:
            docs_test.append(x['content'])
    len1 = len(docs_test)
    print('number of human rights docs is: '+str(len1))
    labels_test = [1]*len1

In [ ]:
# integer encode the documents
encoded_docs_test = t.texts_to_sequences(docs_test)
# pad documents to a max length, compute by calculating the maximum document length
max_length = 20512
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

In [ ]:
# load model
def load_trained_model(weights_path):
    model = build_model()
    model.load_weights(weights_path)
    return model

In [ ]:
train_model = load_trained_model('middle_class.h5')

In [ ]:
train_model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
pred = train_model.predict(padded_docs_test)

In [ ]:
temp = pred[0]>0.1
print(temp.astype(int))
print(labels_test[0])

In [ ]:
print(pred[0]-labels[0])

In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [ ]:
# evaluation
thresh = 0.3
total_precision = 0
total_recall = 0
total_count = 0
for i in range(1000):
    temp_pred = pred[i]>thresh
    temp_pred = list(temp_pred.astype(int))
    temp_actual = labels_test[i]
    
    temp_pred_index = []
    temp_actual_index = []
    
    for k,v in enumerate(temp_pred):
        if v == 1:
            temp_pred_index.append(k)
    
    for k,v in enumerate(temp_actual):
        if v == 1:
            temp_actual_index.append(k)
            
    temp_list = intersection(temp_pred_index,temp_actual_index)
    
    temp_same = len(temp_list)
    if (len(temp_pred_index) != 0 and len(temp_actual_index)!=0 ):
        total_count +=1
        temp_precision = float(temp_same)/len(temp_pred_index)
        temp_recall = float(temp_same)/len(temp_actual_index)
    
    total_precision += temp_precision
    total_recall += temp_recall

In [ ]:
print(total_precision)
print(total_recall)
print(total_count)

In [ ]:
temp_pred = pred[0]>0.1
temp_pred = list(temp_pred.astype(int))
temp_actual = labels_test[0]
temp1 = []
temp2 = []

In [ ]:
for k,v in enumerate(temp_pred):
    if v == 1:
        temp1.append(k)

In [ ]:
for k,v in enumerate(temp_actual):
    if v == 1:
        temp2.append(k)

In [ ]:
len1 = len(temp2)
print(len1)

In [ ]:
len(temp1)

In [ ]:
print(temp1)
print(temp2)

In [ ]:
temp3 = intersection(temp1,temp2)

In [ ]:
print(len(temp3))